## 套件匯入

In [1]:
import pandas as pd
import plotly.express as px
import pickle
import re
import string
import zhon
import text2vec
import jieba
import jieba.analyse
import jieba.posseg as pseg
import plotly.express as px
from text2vec import SearchSimilarity
from docx import Document
from zhon.hanzi import punctuation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## 讀取檔案

In [2]:
f = open('C:/Users/sefx5/Downloads/議程附件4-總綱小組第1-7次諮詢會議紀錄(稿).docx','rb')
document = Document(f)

## 截取會議内容與參與會議人員（本院 / 非本院）

In [3]:
sentence_list,save_content = [],False
for para in document.paragraphs:
    if (save_content) and (para.text != '' or para.text != '\n'):
        sentence_list.append(para.text)
    if para.text == '發言紀要：':
        save_content = True
    if para.text[:5] == '決  議：':
        save_content = False

## 讀入人員名單 Pickle 檔

In [4]:
# 輸入名單

# 將人員列表分爲：
## 1.全體人員 -> final_ppl
## 2.本院人員 -> final_in_ppl
## 3.非本院人員 -> final_out_ppl

infile = open('attendance_list.pkl','rb')
new_dict = pickle.load(infile)
final_in_ppl,final_out_ppl,final_ppl = new_dict[0],new_dict[1],new_dict[2]
print(f'完成寫入 \n->本院人數：{len(final_in_ppl)} \n->非本院人數： {len(final_out_ppl)} \n->Total：{len(final_ppl)}')

完成寫入 
->本院人數：7 
->非本院人數： 116 
->Total：123


## 字典建置

In [ ]:
# 建立 KOL 字典
with open('kol.txt','wb') as f:
    for ppl in final_ppl:
        f.write(f'{ppl} 2000 nr \n'.encode('utf-8'))
    f.close()

## 字典匯入 Jieba

In [5]:
jieba.set_dictionary('./dict.txt.big.txt')
jieba.load_userdict('./kol.txt')

## 觀察資料與剖析

In [6]:
#过滤中英文标点符号、字母、数字
def filter_punc(desstr:list,restr=''):
    pattern = re.compile("[%s]+" % string.punctuation + "|[%s]+" % zhon.hanzi.punctuation + "|[%s]+" % 'a-zA-Z0-9') #匹配中英文符号及字母数字
    filter_ = [pattern.sub(restr,each_sentence) for each_sentence in desstr]
    return filter_

In [7]:
# 清整中英文标点符号、字母、数字
filter_ = filter_punc(sentence_list)
print('會議記錄截取後的句子數量 ：',len(filter_))

會議記錄截取後的句子數量 ： 513


## 人物對發言

In [8]:
# 詞性標記並截取 【名詞 / 動詞 / 形容詞】,將不同發言人的内容進行歸類
speech = {}
coll_perm = False
name = ''

for each_sentence in filter_:
    words = pseg.cut(each_sentence)
    for word,flag in words:
        if word in final_ppl:
            name = word
            speech[name] = []
            coll_perm = True
            break
        if coll_perm:
            if (word.strip() != ''):
                if flag in ['n','nz','a','v','vd']:
                    speech[name].append(word)
print(f'經人物發言歸類後所剩下的人數為{len(speech)}人')
print(f'從原有的{len(final_ppl)}人歸類後剩下{len(speech)}人')

經人物發言歸類後所剩下的人數為78人
從原有的123人歸類後剩下78人


## 檢查被去除人員名單

In [9]:
ignore_list = list(set(final_ppl).difference(set(speech.keys())))
print(ignore_list)
print(f'\n被列入 ignore_list 人員大部分僅爲會議出席者！ \n該人數為{len(ignore_list)}人')

['林侑欣', '黃秀霞', '張箴言', '林宜誠', '郭靜靜', '陳松根', '江俊龍', '鄭慶民', '盧台華', '何孟軒', '彭宬', '顏綠芬', '劉芝芳', '杜思慧', '洪詠善', '張名華', '潘慧玲', '林致宇', '馮喬蘭', '李麗琦', '林秀英', '胡茹萍', '鍾雯豐', '李美娟', '黃啟煌', '曾淑惠', '王俊諺', '陳貴馨', '徐青雲', '政傑', '陳盈儒', '巫彰玫', '許秀菁', '周志宏', '李勤岸', '鄭勝耀', '沂', '葉興華', '張國保', '海樹兒．犮剌拉菲', '鍾屏蘭', '楊曉菁', '劉彥辰', '顏慶祥', '李怡穎']

被列入 ignore_list 人員大部分僅爲會議出席者！ 
該人數為45人


## 人物發言歸類

In [10]:
# 裝歡 Data Frame 格式
name_list = list(speech.keys())
sentence_list = list(speech.values())
df = pd.DataFrame({'name' : name_list , 'sentence' : sentence_list })
df.head()

,name,sentence
0,潘信宏,"[通用, 一般, 不同, 叫, 中文, 教授, 歌, 大量, 啟聰, 打, 文字, 造成, ..."
1,楊秀菁,"[本院, 中心]"
2,吳律德,"[本院, 中心]"
3,李川信,[]
4,李文富,"[原住, 民族, 部定, 是否, 考量, 課, 綱, 做, 本土, 配套措施, 市政府, 教育局]"


In [15]:
df['len_sentence'] = df['sentence'].apply(lambda list_: len(list_))
df = df[df['len_sentence']  > 10]
print(df.shape)
df.head()

(32, 3)


,name,sentence,len_sentence
0,潘信宏,"[通用, 一般, 不同, 叫, 中文, 教授, 歌, 大量, 啟聰, 打, 文字, 造成, ...",18
4,李文富,"[原住, 民族, 部定, 是否, 考量, 課, 綱, 做, 本土, 配套措施, 市政府, 教育局]",12
5,廖浩翔,"[基本, 身心, 特殊, 需求, 教授, 作為, 課綱, 身心, 一般, 理解, 需求, 困...",62
7,黃致誠,"[支持, 放在, 核心, 身為, 原住民, 耆老, 注重, 計畫, 會, 要求, 放在, 附...",14
11,范智鈞,"[切得, 去, 思考, 門, 課, 要, 安排, 小, 不易, 重心, 會, 會, 形成, ...",37


## Article Similarities

In [16]:
# 將斷詞後的句子重組，並放在 1 個 list 中
corpus = [''.join(sentence) for sentence in list(df['sentence'])]
search_sim = SearchSimilarity(corpus=corpus)

# search_sim.get_scores(query=corpus[no]) # 個人發言之相似度向量
# search_sim.get_similarities(query=corpus[no]) # 個人發言之相似度重組句子

In [17]:
# 將重組句子及文章相似度之向量寫入 DataFrame
df['corpus'] = corpus
df['scores'] = df['corpus'].apply(lambda sentence: search_sim.get_scores(sentence))
df = df.reset_index(drop = True)
df.head()

C:\Users\sefx5\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\sefx5\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,name,sentence,len_sentence,corpus,scores
0,潘信宏,"[通用, 一般, 不同, 叫, 中文, 教授, 歌, 大量, 啟聰, 打, 文字, 造成, ...",18,通用一般不同叫中文教授歌大量啟聰打文字造成是故教育部不能應辦,"[52.0136412752401, 0.0, 5.173664308075342, 0.0..."
1,李文富,"[原住, 民族, 部定, 是否, 考量, 課, 綱, 做, 本土, 配套措施, 市政府, 教育局]",12,原住民族部定是否考量課綱做本土配套措施市政府教育局,"[0.0, 23.406695319053355, 2.894136928594631, 0..."
2,廖浩翔,"[基本, 身心, 特殊, 需求, 教授, 作為, 課綱, 身心, 一般, 理解, 需求, 困...",62,基本身心特殊需求教授作為課綱身心一般理解需求困境群體基本身心特殊需求是否符合需求修改程考量可...,"[10.222864022189789, 8.296599871499788, 135.71..."
3,黃致誠,"[支持, 放在, 核心, 身為, 原住民, 耆老, 注重, 計畫, 會, 要求, 放在, 附...",14,支持放在核心身為原住民耆老注重計畫會要求放在附件本土公民,"[0.0, 0.9874676888628043, 4.656881272084541, 4..."
4,范智鈞,"[切得, 去, 思考, 門, 課, 要, 安排, 小, 不易, 重心, 會, 會, 形成, ...",37,切得去思考門課要安排小不易重心會會形成重心本土教育部客語原民會原住民能讓免修本土作法能具有程...,"[1.402081756118654, 4.937338444314022, 8.66067..."


In [18]:
# label 為該名人員對全體人員發言中相似度最高者
# similarities 為該名人員對全體人員發言中的相似程度
df['label'] = df['corpus'].apply(lambda sentence: df['name'].iloc[corpus.index(search_sim.get_similarities(query=sentence)[1])])
df['similarities'] = df['corpus'].apply(lambda sentence: search_sim.get_scores(query=sentence)[corpus.index(search_sim.get_similarities(query=sentence)[1])])
print(df.shape)
df.head()

(32, 7)


,name,sentence,len_sentence,corpus,scores,label,similarities
0,潘信宏,"[通用, 一般, 不同, 叫, 中文, 教授, 歌, 大量, 啟聰, 打, 文字, 造成, ...",18,通用一般不同叫中文教授歌大量啟聰打文字造成是故教育部不能應辦,"[52.0136412752401, 0.0, 5.173664308075342, 0.0...",張明旭,5.290268
1,李文富,"[原住, 民族, 部定, 是否, 考量, 課, 綱, 做, 本土, 配套措施, 市政府, 教育局]",12,原住民族部定是否考量課綱做本土配套措施市政府教育局,"[0.0, 23.406695319053355, 2.894136928594631, 0...",顏國樑,7.252580
2,廖浩翔,"[基本, 身心, 特殊, 需求, 教授, 作為, 課綱, 身心, 一般, 理解, 需求, 困...",62,基本身心特殊需求教授作為課綱身心一般理解需求困境群體基本身心特殊需求是否符合需求修改程考量可...,"[10.222864022189789, 8.296599871499788, 135.71...",陳志勇,19.010635
3,黃致誠,"[支持, 放在, 核心, 身為, 原住民, 耆老, 注重, 計畫, 會, 要求, 放在, 附...",14,支持放在核心身為原住民耆老注重計畫會要求放在附件本土公民,"[0.0, 0.9874676888628043, 4.656881272084541, 4...",賴慧珠,6.128988
4,范智鈞,"[切得, 去, 思考, 門, 課, 要, 安排, 小, 不易, 重心, 會, 會, 形成, ...",37,切得去思考門課要安排小不易重心會會形成重心本土教育部客語原民會原住民能讓免修本土作法能具有程...,"[1.402081756118654, 4.937338444314022, 8.66067...",張明旭,17.559774


In [19]:
# 假設以這些人員名單為相似度高的主軸
num_main = len(df['label'].unique())
print(f"作爲發言相似度高之核心人過代表有 {num_main} 人")
df['label'].value_counts()[:9]

作爲發言相似度高之核心人過代表有 16 人


呂明蓁     4
陳張培倫    4
張明旭     4
林家琿     3
陳志勇     3
許綉敏     2
范智鈞     2
林秀珍     2
李立彬     1
Name: label, dtype: int64

In [20]:
# 以陳張培倫爲例，將與陳張培倫高度相似的組群去除斷詞列表，並以交集關鍵字呈現
top_1_subset_name = list(df[df['label'] == '陳張培倫']['name'])
top_1_subset_stopword = list(df[df['label'] == '陳張培倫']['sentence'])
main = df[df['name'] == '陳張培倫']['sentence'].values.tolist()[0]
for no in range(len(top_1_subset_name)):
    print(f'【{top_1_subset_name[no]}】 對陳張培倫之交集為 : {set(main).intersection(set(top_1_subset_stopword[no]))}',end = '\n')
    print('='*120)
print(f'【陳張培倫】 之關鍵字為 ： {main}')

【陳志勇】 對陳張培倫之交集為 : {'需', '寒暑假', '本土', '原住民', '不同', '可', '部定', '思考', '考量', '是否', '列入', '需求'}
【林秀珍】 對陳張培倫之交集為 : {'中心', '必修', '新', '部定', '考量', '普通型', '寒暑假', '能', '原住民', '有', '本院', '住民', '本土', '附件', '基本法', '思考', '陸', '原住', '部分', '民族', '缺乏', '科目', '可', '草案', '是否'}
【顏國樑】 對陳張培倫之交集為 : {'中心', '附件', '草案', '考量', '原住民', '本院'}
【林家琿】 對陳張培倫之交集為 : {'住民', '民族', '是否', '寒暑假', '本土', '情形', '附件', '新', '陸', '考量', '草案'}
【陳張培倫】 之關鍵字為 ： ['原住民', '本土', '民族', '考科', '列入', '考科', '原住民', '升高', '有', '草案', '附件', '陸', '本土', '新', '住民', '本土', '客', '民族', '平埔族', '平埔族', '歸為', '原住', '民族', '基本法', '思考', '平埔族', '草案', '附件', '陸', '本土', '新', '住民', '部分', '民族', '寒暑假', '缺乏', '可', '需求', '考量', '民族', '是否', '寒暑假', '草案', '附件', '陸', '普通型', '修', '科目', '最低', '人', '情形', '特殊', '足以', '降低', '下限', '人', '端', '能', '限制', '考量', '草案', '附件', '原住民', '部定', '必修', '一般', '涉及', '受', '考量', '民族', '民族', '考科', '不同', '有', '需', '本院', '中心']


## Doc2Vec

In [ ]:
rows = StandardScaler().fit_transform(list(df['scores']))
df_doc2vec = pd.DataFrame(rows,columns = list(df['name']))
print(df_doc2vec.shape)
df_doc2vec.head()

## PCA（Dimension Reduction）

In [ ]:
x = df_doc2vec.loc[:,:].values
pca = PCA(n_components = 2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents,columns = ['feature_1', 'feature_2'])
finalDf = pd.concat([principalDf, df[['name']]], axis = 1)
print(finalDf.shape)
finalDf.head()

In [ ]:
# 顔色設定(本院 / 非本院)
finalDf['color'] = finalDf['name'].apply(lambda name:"1" if name in final_in_ppl else "0")

In [ ]:
# 顔色設定（依 label 上色）
case_own = {}
label_list = list(df['label'])
name_list = list(finalDf['name'])

for each in df['label'].unique():
    case_own[str(name_list.index(each))] = each
    
for case in case_own.keys():
    label_list[int(case)] = case_own[case]
    
finalDf['color_label'] = label_list

In [ ]:
pca.explained_variance_ratio_

In [ ]:
fig = px.scatter(                                             \
    finalDf, x="feature_1", y="feature_2",                    \
    title = 'PCA：以全體人員相似度作爲特徵截取之觀測（散點圖）',  \
    text = "name",                                            \
    color = "color_label"
)

fig.show()

## 代表人物之關鍵字分析

In [ ]:
X_MIN = -3.13
X_MAX = 0
Y_MIN = -1
Y_MAX = 1

name_aly_list = finalDf[((finalDf['feature_1'] > X_MIN) & (finalDf['feature_1'] < X_MAX)) & ((finalDf['feature_2'] > Y_MIN) & (finalDf['feature_2'] < Y_MAX))]['name'].values.tolist()

In [ ]:
name_aly_list

In [ ]:
test1 = df[~df['name'].isin(name_aly_list)].reset_index(drop = True)['name'].values.tolist()

In [ ]:
test1